In [1]:
import numpy as numpy
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import f1_score

In [3]:
pip install torchsummary

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.2 -> 25.0
[notice] To update, run: python.exe -m pip install --upgrade pip


In [4]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
from torchsummary import summary

In [10]:
import torchvision.transforms as transforms
from torchvision.datasets import MNIST
from torch.utils.data import DataLoader

In [9]:
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
print('device:', device)

device: cpu


In [14]:
# 데이터 로드하기
transform = transforms.Compose([transforms.ToTensor(), transforms.Normalize((0.5,), (0.5,))])

train_dataset = MNIST(root='./mnistdata', train=True, download=True, transform=transform)
test_dataset = MNIST(root='./mnistdata', train=False, download=True, transform=transform)

train_loader = DataLoader(dataset=train_dataset, batch_size=64, shuffle=True)
test_loader = DataLoader(dataset=test_dataset, batch_size=64, shuffle=False)

Failed to download (trying next):
HTTP Error 404: Not Found



100%|██████████| 9912422/9912422 [00:02<00:00, 3899717.88it/s]


Extracting ./mnistdata\MNIST\raw\train-images-idx3-ubyte.gz to ./mnistdata\MNIST\raw

Failed to download (trying next):
HTTP Error 404: Not Found



100%|██████████| 28881/28881 [00:00<00:00, 153738.18it/s]


Extracting ./mnistdata\MNIST\raw\train-labels-idx1-ubyte.gz to ./mnistdata\MNIST\raw

Failed to download (trying next):
HTTP Error 404: Not Found



100%|██████████| 1648877/1648877 [00:01<00:00, 1404967.36it/s]


Extracting ./mnistdata\MNIST\raw\t10k-images-idx3-ubyte.gz to ./mnistdata\MNIST\raw

Failed to download (trying next):
HTTP Error 404: Not Found



100%|██████████| 4542/4542 [00:00<?, ?it/s]

Extracting ./mnistdata\MNIST\raw\t10k-labels-idx1-ubyte.gz to ./mnistdata\MNIST\raw



In [ ]:
## 이건, 데이터 로더에서 또 H.P 변경이 필요한 경우를 고려하여 코드 수정!
from torchvision import transforms, datasets
from torch.utils.data import DataLoader
from torchvision.datasets import MNIST

def create_data_loaders(batch_size):
    transform = transforms.

In [18]:
class VGG16(nn.Module):
    def __init__(self, num_classes=10):
        super(VGG16, self).__init__()
        self.features = nn.Sequential(
            nn.Conv2d(1, 64, kernel_size=3, stride=1, padding=1),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=2, stride=2, padding=0),
            nn.Conv2d(64, 128, kernel_size=3, stride=1, padding=1),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=2, stride=2, padding=0),
            nn.Conv2d(128, 256, kernel_size=3, stride=1, padding=1), 
            nn.ReLU(inplace=True),
            nn.Conv2d(256, 256, kernel_size=3, stride=1, padding=1),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=2, stride=2, padding=0)
        )

        self.classifier = nn.Sequential(
            nn.Linear(256 * 3 * 3, 4096),
            nn.ReLU(inplace=True),
            nn.Dropout(),
            nn.Linear(4096, 4096),
            nn.Dropout(),
            nn.Linear(4096, num_classes)
        )
    
    def forward(self, x):
        x = self.features(x)
        x = x.view(x.size(0), -1)
        x = self.classifier(x)
        return x

In [19]:
model = VGG16(num_classes=10)

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

In [20]:
summary(model, input_size=(1, 28, 28))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1           [-1, 64, 28, 28]             640
              ReLU-2           [-1, 64, 28, 28]               0
         MaxPool2d-3           [-1, 64, 14, 14]               0
            Conv2d-4          [-1, 128, 14, 14]          73,856
              ReLU-5          [-1, 128, 14, 14]               0
         MaxPool2d-6            [-1, 128, 7, 7]               0
            Conv2d-7            [-1, 256, 7, 7]         295,168
              ReLU-8            [-1, 256, 7, 7]               0
            Conv2d-9            [-1, 256, 7, 7]         590,080
             ReLU-10            [-1, 256, 7, 7]               0
        MaxPool2d-11            [-1, 256, 3, 3]               0
           Linear-12                 [-1, 4096]       9,441,280
             ReLU-13                 [-1, 4096]               0
          Dropout-14                 [-

In [21]:
## 자, 이제 완디비를 설치해 봅시다잇
import wandb

## 개인 API 먼저!
wandb.login()

Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: junylee00 (junylee00-chonnam-national-university) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


True

In [22]:
wandb.init(project='Weight Biases Beginner with MNIST',
           name='VGG16 Basic')

wandb.watch(model)
## 학습 그래프, GPU 시스템 등 로딩 ㄱㄱ => 하단 링크에 기록!!

wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.


In [34]:
# Training Loop
def train_one_epoch(model, train_loader, optimizer, criterion):
    model.train()
    for images, labels in train_loader:
        images, labels = images.to(device), labels.to(device)
        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()     # Back Propagation
        optimizer.step()
    return loss.item()

# Validation 과정(with 테스트 데이터셋)
def validate_one_epoch(model, test_loader, criterion):
    model.eval()
    total_loss = 0
    total_metrics = {'accuracy': 0, 'f1_score': 0}
    num_batches = 0

    with torch.no_grad():
        for images, labels in test_loader:
             images, labels = images.to(device), labels.to(device)
             outputs = model(images)
             loss = criterion(outputs, labels)
             total_loss += loss.item()

             _, predicted = torch.max(outputs.data, 1)
             total = labels.size(0)
             correct = (predicted == labels).sum().item()
             accuracy = correct / total
             predicted_cpu = predicted.cpu()
             labels_cpu = labels.cpu()
             f1 = f1_score(labels_cpu, predicted_cpu, average='macro')

             total_metrics['accuracy'] += accuracy
             total_metrics['f1_score'] += f1
             num_batches += 1
    
    avg_loss = total_loss / num_batches
    avg_metrics = {k: v / num_batches for k, v in total_metrics.items()}

    # 한 에포크마다 validation 하고 기록!!
    wandb.log({
        'val_accuracy': avg_metrics['accuracy'],
        'val_f1_score': avg_metrics['f1_score']
    })

    return avg_loss

In [35]:
def main(model, train_loader, test_loader, optimizer, criterion, num_epochs=10):
    # 에포크 마다 train, valid 고고
    for epoch in range(num_epochs):
        train_loss = train_one_epoch(model, train_loader, optimizer, criterion)
        val_loss = validate_one_epoch(model, test_loader, criterion)

        # 에포크 마다 train loss, val loss 기록 ㄱㄱㄱ
        wandb.log({
            'train_loss': train_loss,
            'val_loss': val_loss,
            'epoch': epoch
        })

        print(f"Epoch {epoch+1}/{num_epochs}, Train Loss: {train_loss:.4f}, Val Loss: {val_loss:.4f}")

In [36]:
model = model.to(device)
main(model, train_loader, test_loader, optimizer, criterion, num_epochs=10)

Epoch 1/10, Train Loss: 0.0076, Val Loss: 0.0341
Epoch 2/10, Train Loss: 0.1742, Val Loss: 0.0424
Epoch 3/10, Train Loss: 0.0029, Val Loss: 0.0483
Epoch 4/10, Train Loss: 0.0685, Val Loss: 0.0741
Epoch 5/10, Train Loss: 0.0001, Val Loss: 0.0467
Epoch 6/10, Train Loss: 0.0003, Val Loss: 0.0455
Epoch 7/10, Train Loss: 0.0000, Val Loss: 0.0461
Epoch 8/10, Train Loss: 0.1756, Val Loss: 0.0493
Epoch 9/10, Train Loss: 0.0350, Val Loss: 0.0533
Epoch 10/10, Train Loss: 0.0000, Val Loss: 0.0465


In [ ]:
wandb.finish()